
## Descripción del prob_2lema

Una compañía petrolera quiere decidir cómo operar durante los próximos años en una zona en la que posee 4 yacimientos petrolíferos ($A$, $B$, $C$ y $D$). 

Estos yacimientos tienen una capacidad máxima anual de extracción dada por:

| Yacimiento | Extracción
| --- | ---
| $A$ | 250
| $B$ | 312.5
| $C$ | 162.5
| $D$ | 375

Donde la extracción se da en miles de barriles. 

Los yacimientos usan 2 tipos de tecnología de extracción diferentes. $A$ y $D$ son del tipo $T_1$, mientras que $B$ y $C$ son del tipo $T_2$. Cada tecnología le supone a la compañía unos gastos anuales fijos por mantener en operación cada yacimiento de esa tecnología:

| Tipo | Anualidad |
| --- | --- |
| $T_1$ | 5 |
| $T_2$ | 4 |

Donde las anualidades se dan en millones de €.

Una vez extraído petróleo de cada yacimiento, la compañía necesita **refinarlo antes de poderlo poner a la venta**. Para ello, ha decidido crear una métrica propia para medir la pureza del crudo refinado, y se ha impuesto como objetivo a 5 años vista que **la pureza del que termine refinando cada año debe ser**:

| Año | Pureza |
| --- | --- |
| 1 | 0.9 |
| 2 | 0.8 |
| 3 | 1.2 |
| 4 | 0.6 |
| 5 | 1.0 |

Mientras que la pureza del que obtiene de cada yacimiento individual **sin refinar** es:

| Yacimiento | Pureza
| --- | ---
| A | 1.25
| B | 0.875
| C | 1.875
| D | 0.625

La métrica está diseñada de tal forma que **se puede asumir que, cuando se mezclan petróleos de distinta pureza sin refinar, dicha pureza se combina linealmente, con la salvedad de que se degrada un 20% durante la mezcla**. Por ejemplo, al mezclar 100 barriles de petróleo extraído sin refinar del yacimiento A con otros 100 del yacimiento D, la pureza $P$ del petróleo resultante cumple que:

$200*P = 0.8*(100*1.25 + 100*0.625) = 0.8*(125 + 62.5) = 0.8*187.5 = 150$

Luego $P = 150/200 = 0.75$, mientras que si no se perdiera ese 20% de pureza en el proceso, el resultado habría sido de $P = (1.25 + 0.625)/2 = 0.9375$.

**Una vez refinado, el barril de petróleo la compañía lo vende a 80 €.**

Lógicamente, **la compañía desea maximizar su beneficio, pero garantizando que se cumple el requisito de pureza y que no sobreexplota los yacimientos**.


## Parte A: optimización lineal

Formular el prob_2lema exclusivamente para el primer año (es decir, qué debe hacer la compañía para el año 1). Se puede suponer lo siguiente:
* **El yacimiento $B$ todavía no está en disposición de utilizarse**.
* **Para los 3 restantes ($A$, $C$ y $D$), la compañía ya ha decidido de antemano que se exploten, y ha reservado el dinero necesario para ello** (es decir, que es como si ya hubiera pagado las anualidades; 5 millones de € para $A$, otros 5 millones para $D$, y 4 millones para $C$).

No obstante, todavía debe decidir cuánto petróleo se debe extraer de cada uno de estos 3 yacimientos. Se pide:
1. Formular el prob_2lema primal.
2. Resolverlo mediante PuLP, especificando:
    * Cuántos barriles se deben extraer de $A$, de $C$ y de $D$.
    * ¿Hay algún yacimiento que se decida no usar a su máximo rendimiento?
    * Qué beneficio obtiene la compañía con ello (es decir, ingresos por los barriles refinados menos los pagos de las anualidades ya referidas).
3. Formular el prob_2lema dual y resolverlo también. ¿Se obtiene la misma solución? ¿Por qué sí o por qué no?

- Formulación del problema: <br>
    $Y = \{A, C, D\}$ (yacimientos).<br>
    $x_y$ = barriles extraídos del yacimineto $y$$ <br>
    $q_y$ = capacidad de extracción del yacimiento $y$ <br>
    $p_y$ = pureza de la extracción del yacimiento $y$ <br>
    $c_y$ = coste de explotación del yacimiento $y$ <br>
    $v$ = precio de venta del barril<br>
    $t$ = pureza final<br>

    La función a maximizar:

    $max \quad \displaystyle\sum_{y \in Y}x_y * v - c_y$

    Restricciones:

    $x_y \leq q_y \quad  \forall y$ <br><br>
    $t \displaystyle\sum_{y \in Y}x_y = 0.8 * \displaystyle\sum_{y \in Y}(p_y * x_y) \quad$
    

In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
yacimientos_año1 = ["A", "C", "D"]
capacidad_extraccion_año1 = pd.Series(index = yacimientos_año1, data =  [250, 162.5, 375])
pureza_final_año1 = 0.9
pureza_extraccion_por_yacimiento_año1 = pd.Series(index = yacimientos_año1, data = [1.25, 1.875, 0.625])
coste_yacimiento_año1 = pd.Series(index = yacimientos_año1, data = [5, 4, 5]).mul(1000000)
venta_barril_año1 = 80 * 1000

In [3]:
# Crear problema de maximización
prob = LpProblem("Yacimientos_LM", LpMaximize)

# Crear variables de asientos de cada tarifa 
barriles_año1 = LpVariable.dicts("Barriles_Extraídos", indices = yacimientos_año1, lowBound = 0, cat = LpContinuous)

prob += lpSum([barriles_año1[yacimiento] * venta_barril_año1 -  coste_yacimiento_año1[yacimiento]  for yacimiento in yacimientos_año1])    

for yacimiento in yacimientos_año1:
    prob += barriles_año1[yacimiento] <= capacidad_extraccion_año1.at[yacimiento]

prob += (0.8 * lpSum([barriles_año1[yacimiento] * pureza_extraccion_por_yacimiento_año1[yacimiento] for yacimiento in yacimientos_año1]) 
        == 
        pureza_final_año1 * lpSum([barriles_año1[yacimiento] for yacimiento in yacimientos_año1]))

status = prob.solve()
print("Status: ", LpStatus[status])

for v in prob.variables():
  print(v.name, " = ", v.varValue)

print("Beneficio esperado: ", round(value(prob.objective), 2), "€")

Status:  Optimal
Barriles_Extraídos_A  =  250.0
Barriles_Extraídos_C  =  162.5
Barriles_Extraídos_D  =  306.25
Beneficio esperado:  43500000.0 €


## Parte B: optimización entera

Aquí se pide generalizar la formulación del caso anterior para cubrir los 5 años, de forma que **año a año se decida**:
* **Qué yacimientos de los 4 se deben explotar y cuáles no**.
* **En caso de explotarse, cuánto se debe extraer de cada uno**.

Además, algunas simplificaciones de la Parte A ya no son aplicables. En concreto:
* **Todos los yacimientos están disponibles desde el año 1** ($B$ inclusive).
* **No se ha pagado todavía ninguna anualidad** (porque primero debe decidirse para cada año qué yacimientos se explotan).

El resto sí que se mantienen, en especial el que **para cada año debe cumplirse de forma exacta con el requisito de pureza**.

No sólo eso, sino que la compañía establece algunas restricciones adicionales:
* **Cada año como mucho se pueden explotar 3 yacimientos de los 4** (por limitaciones de personal y medioambientales).
* **Aunque un yacimiento no se explote en un determinado año, deben irse pagando todas las anualidades correspondientes si en algún año posterior sí que se explota**. Por ejemplo, si el yacimiento $A$ decibe explotarse sólo el año 1 y el 5, también deben pagarse sus anualidades de los años 2, 3 y 4.
* **Una excepción a lo anterior es si se desmantela un yacimiento por falta de rentabilidad, en cuyo caso ya no se paga ninguna anualidad desde entonces, pero de ese año (inclusive) en adelante ya no puede utilizarse más**. Por ejemplo, si el yacimiento $A$ no se explotara el año 1, sí que se explotara en el año 2, y decidiera desmantelarse en el año 3, se pagaría su anualidad para el año 1 (porque se va a usar en el 2) y lógicamente la del año 2 también, pero no se pagaría nada por él ni en el 3, ni en el 4, ni el 5 (eso sí, en ninguno de ellos podría extraerse nada de él, al estar ya desmantelado).
* **Por efectos de inflación y de competidores en el mercado, cada año que pasa se reduce un 10% el precio al que la compañía vende el petróleo. No obstante, también se reducen en ese mismo 10% las anualidades**. Por ejemplo, si el año 1 el petróleo se vende a 80 € y la anualidad del yacimiento $A$ es de 5 millones de euros, en el año 2 es como si lo vendiera a $80/1.1$ €, pero a cambio la anualidad pasa a ser de $5/1.1$ millones de €, en el año 3 sería $80/1.1^2$ y $5/1.1^2$ respectivamente, y así.

Se pide:
1. Formular el prob_2lema atendiendo a estas nuevas restricciones.
2. Resolverlo mediante PuLP, especificando para cada uno de los 5 años en plan detallado:
    * De qué yacimientos se extraen barriles (y cuántos).
    * De cuáles no se extrae nada (pero se mantienen operativos).
    * Cuáles se desmantelan (si es que hay alguno que se desmantele).
3. Responder también a las siguientes preguntas sobre el resultado global del plan:
    * ¿Cuál es el beneficio esperado por la compañía a lo largo de los 5 años?
    * ¿Sale para el año 1 lo mismo que en la Parte A? ¿Por qué sí o por qué no?

In [4]:
yacimientos = ["A", "B", "C", "D"]
años = ["Y1", "Y2", "Y3", "Y4", "Y5"]
capacidad_extraccion = pd.Series(index = yacimientos, data = [250, 312.5, 162.5, 375])
pureza_final_por_año = pd.Series(index = años, data = [0.9, 0.8, 1.2, 0.6, 1])
pureza_extraccion_por_yacimiento = pd.Series(index = yacimientos, data = [1.25, 0.875, 1.875, 0.625])
coste_yacimiento_año = pd.DataFrame(index = yacimientos, columns = años, data = np.array([5, 4, 4, 5]*5).reshape((4, 5), order = 'F'))
inflacion_año = pd.Series(index = años, data = [1 / 1.1**n for n in range(0, 5)])
coste_yacimiento_año = coste_yacimiento_año.mul(inflacion_año, axis = 1).mul(1000000)
venta_barril = pd.Series(index = años, data = inflacion_año.mul(80).mul(1000))

In [5]:
# Crear problema de maximización
prob = LpProblem("Yacimientos_MLP", LpMaximize)

# Crear variables de asientos de cada tarifa 
barriles = LpVariable.dicts("Barriles_Extraídos", indices = (yacimientos, años), lowBound = 0, cat = LpContinuous)
disponibles = LpVariable.dicts("Yacimiento_Disponible", indices = (yacimientos, años), cat =LpBinary)
en_uso = LpVariable.dicts("Yacimiento_en_Uso", indices = (yacimientos, años), cat = LpBinary)  

objetivo = []
for año in años:
    objetivo += lpSum([barriles[yacimiento][año] * venta_barril[año] - disponibles[yacimiento][año] * coste_yacimiento_año.at[yacimiento, año]  for yacimiento in yacimientos])    
prob += objetivo

for año in años:
    for yacimiento in yacimientos:
        prob += barriles[yacimiento][año] <= capacidad_extraccion.at[yacimiento]

for año in años:
    for yacimiento in yacimientos:
        prob += barriles[yacimiento][año] <= (capacidad_extraccion.at[yacimiento] * en_uso[yacimiento][año])

for año in años:
    for yacimiento in yacimientos:
        prob += en_uso[yacimiento][año] <= disponibles[yacimiento][año]

for yacimiento in disponibles:
    keyList = sorted(disponibles[yacimiento].keys())
    for id_year, year in enumerate(keyList):
        if id_year != 0:
            prob += disponibles[yacimiento][keyList[id_year - 1]] >= disponibles[yacimiento][keyList[id_year]]

for año in años:
    prob+= (lpSum([en_uso[yacimiento][año] for yacimiento in yacimientos]) <= 3)

for año in años:
    prob += (0.8 * lpSum([barriles[yacimiento][año] * pureza_extraccion_por_yacimiento[yacimiento] for yacimiento in yacimientos]) 
            == 
            pureza_final_por_año[año] * lpSum([barriles[yacimiento][año]  for yacimiento in yacimientos]))

status = prob.solve()
print("Status: ", LpStatus[status])

for v in prob.variables():
  print(v.name, " = ", v.varValue)

print("Beneficio esperado: ", round(value(prob.objective), 2), "€")


Status:  Optimal
Barriles_Extraídos_A_Y1  =  250.0
Barriles_Extraídos_A_Y2  =  0.0
Barriles_Extraídos_A_Y3  =  243.75
Barriles_Extraídos_A_Y4  =  15.625
Barriles_Extraídos_A_Y5  =  250.0
Barriles_Extraídos_B_Y1  =  0.0
Barriles_Extraídos_B_Y2  =  312.5
Barriles_Extraídos_B_Y3  =  0.0
Barriles_Extraídos_B_Y4  =  312.5
Barriles_Extraídos_B_Y5  =  270.83333
Barriles_Extraídos_C_Y1  =  162.5
Barriles_Extraídos_C_Y2  =  162.5
Barriles_Extraídos_C_Y3  =  162.5
Barriles_Extraídos_C_Y4  =  0.0
Barriles_Extraídos_C_Y5  =  162.5
Barriles_Extraídos_D_Y1  =  306.25
Barriles_Extraídos_D_Y2  =  275.0
Barriles_Extraídos_D_Y3  =  0.0
Barriles_Extraídos_D_Y4  =  375.0
Barriles_Extraídos_D_Y5  =  0.0
Yacimiento_Disponible_A_Y1  =  1.0
Yacimiento_Disponible_A_Y2  =  1.0
Yacimiento_Disponible_A_Y3  =  1.0
Yacimiento_Disponible_A_Y4  =  1.0
Yacimiento_Disponible_A_Y5  =  1.0
Yacimiento_Disponible_B_Y1  =  1.0
Yacimiento_Disponible_B_Y2  =  1.0
Yacimiento_Disponible_B_Y3  =  1.0
Yacimiento_Disponible_B_Y4  

Los barriles que se extraen en cada yacimineto por año: 

| Yacimiento | Año 1 | Año 2 | Año 3 | Año 4 | Año 5 
| --- | --- | --- | --- | --- | ---
| A | 250 | 0 | 243.75 | 15.625 | 250
| B | 0 | 312.5 | 0 | 312.5 | 270.83
| C | 162.5 | 162.5 | 162.5 | 0 | 162.5
| D | 306.25 | 275 | 0 | 375 | 0



No se desmantela ningún yacimiento hasta el 5º año que se desmantela el D, ya que no se va a utilizar y produciría un gasto innecesario.


El beneficio esperado por la compañia es de: 146.861.974,18 €.

Para el primer año obtenemos los mismo resultados que en la primera parte ya que en está ocasión tampoco se va a utilizar B.